# Tech Intro

## Main tables

In [654]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [655]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[3] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [656]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 01:52:14.315583 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 01:52:14.316124 48928 sched.cpp:241] Version: 1.11.0
I1120 01:52:14.316300 10528 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 01:52:14.316367 10528 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 01:52:14.317867 10538 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75613


## Read the spec-file from Hadoop and show some data sample

In [657]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+-----------------------------------------------------+-----+------+----------+----------------------------------------------------------------+---------+
|id     |name                                                 |cost |deal  |updated   |note                                                            |type_cost|
+-------+-----------------------------------------------------+-----+------+----------+----------------------------------------------------------------+---------+
|1000005|Планируемый размер вознаграждения                    |3 %  |107949|1586909450|Планируемый размер вознаграждения                               |null     |
|1000006|Планируемый размер дополнительного вознаграждения    |null |107949|null      |Планируемый размер дополнительного вознаграждения               |null     |
|1000008|Фактически полученная сумма вознаграждения           |null |107949|null      |Фактически полученная сумма вознаграждения                      |null     |
|1000019|Сумма аванса 

## Show columns' properties

In [658]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- deal: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- note: string (nullable = true)
 |-- type_cost: string (nullable = true)



## Total rows (count)

In [659]:
eda_df.count()

2863029

## Simple func to execute and show the result of the SQL-instruction

In [660]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [661]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [662]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|2863029 |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [663]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,name,cost,deal,updated,note,type_cost
0,0,0,2161627,0,2215399,0,2859151


In [664]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,name,cost,deal,updated,note,type_cost
0,0,0,7578,0,0,0,0


In [665]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,name,cost,deal,updated,note,type_cost
0,0,0,2272,0,0,3438,0


In [666]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
name,0,0,0
cost,2161627,7578,2272
deal,0,0,0
updated,2215399,0,0
note,0,0,3438
type_cost,2859151,0,0


In [667]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [668]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
name,0.000,0.000,0.000
cost,75.501,0.265,0.079
deal,0.000,0.000,0.000
updated,77.380,0.000,0.000
note,0.000,0.000,0.120
type_cost,99.865,0.000,0.000


### SQL-instructions

In [669]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|2863029           |
+------------------+



+--------------------+
|count(DISTINCT name)|
+--------------------+
|27                  |
+--------------------+

+--------------------+
|count(DISTINCT cost)|
+--------------------+
|24578               |
+--------------------+

+--------------------+
|count(DISTINCT deal)|
+--------------------+
|190817              |
+--------------------+



+-----------------------+
|count(DISTINCT updated)|
+-----------------------+
|643462                 |
+-----------------------+

+--------------------+
|count(DISTINCT note)|
+--------------------+
|27                  |
+--------------------+

+-------------------------+
|count(DISTINCT type_cost)|
+-------------------------+
|2                        |
+-------------------------+



In [670]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [671]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+------+
|id    |
+------+
|999999|
|999998|
|999997|
|999996|
|999995|
|999994|
|999993|
|999992|
|999991|
|999990|
|99999 |
|999989|
|999988|
|999987|
|999986|
|999985|
|999984|
|999983|
|999982|
|999981|
+------+
only showing top 20 rows

None


+-------+
|id     |
+-------+
|1000   |
|10000  |
|100000 |
|1000000|
|1000001|
|1000002|
|1000003|
|1000004|
|1000005|
|1000006|
|1000007|
|1000008|
|1000009|
|100001 |
|1000010|
|1000011|
|1000012|
|1000013|
|1000014|
|1000015|
+-------+
only showing top 20 rows



+--------------------------------------------------------------+
|name                                                          |
+--------------------------------------------------------------+
|Фактический размер вознаграждения по отчету                   |
|Фактический размер вознаграждения                             |
|Фактически полученная сумма вознаграждения                    |
|Фактически полученная сумма                                   |
|Уточненная (измененная) стоимость по агентскому договору      |
|Сумма задатка по соглашению предварительного договора         |
|Сумма аванса по соглашению, вносимого в агентство недвижимости|
|Сумма аванса по договору о намерениях                         |
|Сумма аванса между продавцом и покупателем                    |
|Стоимость при заключении агентского договора                  |
|Стоимость при заключении АД                                   |
|Стоимость по доп. соглашению к договору                       |
|Стоимость по доп. соглаш

+----------------------------------------------------------------+
|name                                                            |
+----------------------------------------------------------------+
|Cумма аванса по ДНПН                                            |
|Cумма аванса по договору о намерениях                           |
|Cумма задатка (аванса) по соглашению к предварительному договору|
|Cумма задатка по соглашению                                     |
|Вознаграждение риэтора                                          |
|Кадастровая стоимость                                           |
|Планируемый размер вознаграждения                               |
|Планируемый размер дополнительного вознаграждения               |
|Реальная (фактическая) стоимость                                |
|Реальная стоимость                                              |
|Стоимость (формальная) по договору                              |
|Стоимость неотделимых улучшений                              

+------+
|deal  |
+------+
|100   |
|1000  |
|10000 |
|100000|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|10001 |
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
+------+
only showing top 20 rows

+----------+
|updated   |
+----------+
|1666809561|
|1666808698|
|1666808685|
|1666808025|
|1666808016|
|1666799652|
|1666798762|
|1666798120|
|1666796221|
|1666796213|
|1666796115|
|1666796068|
|1666796057|
|1666795531|
|1666795526|
|1666795518|
|1666795343|
|1666795267|
|1666795261|
|1666795257|
+----------+
only showing top 20 rows

None


+----------+
|updated   |
+----------+
|null      |
|1489643173|
|1489643189|
|1489643198|
|1489643204|
|1489644343|
|1489644359|
|1489644390|
|1489646326|
|1489646368|
|1489648439|
|1489648735|
|1489649325|
|1489649335|
|1489650692|
|1489650703|
|1489655433|
|1489655444|
|1489656176|
|1489656187|
+----------+
only showing top 20 rows

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|note                                                                                                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Фактический размер 

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [653]:
spark.stop()

I1120 01:51:42.432068 10390 sched.cpp:2174] Asked to stop the driver
I1120 01:51:42.432529 10528 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75610
